In [3]:
import warnings
import subprocess
from selenium import webdriver
import time
import os


start = time.time()

warnings.filterwarnings('ignore')
checkfilepath = r"C:\Users\016434613\Downloads\progress_dt_tasks_master_report.xlsx"
exists = False

# This sentence takes care of deleting old mater report on downloads folder
subprocess.run("del /f C:\\Users\\016434613\\Downloads\\progress_dt_tasks_master_report.xlsx", shell=True, capture_output=True)

mydriver = webdriver.Chrome("chromedriver.exe")
mydriver.get("https://progress.us1a.cirrus.ibm.com/api/rpt/mst/1588760663192/lf/bd3hd8xaj/0/1588760727599")
while not exists:
    time.sleep(1)
    exists = os.path.exists(checkfilepath)
mydriver.close()

end = time.time()
mins = (end-start)/60.00
if mins < 1:
    mins = end-start
    minsstr = "Report created in: {:.2f} seconds".format(mins) 
    print(minsstr)
else:
    minsstr = "Report created in: {:.2f} minutes".format(mins)
    print(minsstr)

Report created in: 1.20 minutes


In [4]:
import pandas as pd
import numpy as np
import warnings
import glob
import re
import time
from fast_reports import *

start = time.time()

warnings.filterwarnings('ignore')

filepath_ = r"C:\Users\016434613\Downloads\progress_dt_tasks_details_report.xlsx"

df = pd.read_excel(filepath_,sheet_name="Tasks")
df = df.rename(columns={'ROBOT NUMBER':'ROBOT_NUMBER'})

df_lindo = df.loc[:,['ORIGIN RELEASE','STATUS','ROBOT_NUMBER','TITLE','AUTOMATION TYPE','REGIONS','MANUAL EXECUTION DURATION (hours per year)','CURRENT VERSION ID']]
df_lindo = df_lindo.rename(columns={'MANUAL EXECUTION DURATION (hours per year)':'SAVING HS','CURRENT VERSION ID':'VERSION ID'})

df4 = df_lindo
df4['ORIGIN RELEASE'] = df4['ORIGIN RELEASE'].apply(lambda x: str(x))
df4['QUARTER'] = df4['ORIGIN RELEASE'].apply(lambda x: x[4:7]+x[11:15])

df4['QUARTER'] = df4['QUARTER'].apply(lambda x: '1Q21' if x == 'Mar2021' else '1Q21' if x == 'Feb2021' else '1Q21' if x == 'Jan2021' else x)
df4['QUARTER'] = df4['QUARTER'].apply(lambda x: '2Q21' if x == 'Jun2021' else '2Q21' if x == 'May2021' else '2Q21' if x == 'Apr2021' else x)
df4['QUARTER'] = df4['QUARTER'].apply(lambda x: '3Q21' if x == 'Sep2021' else '3Q21' if x == 'Aug2021' else '3Q21' if x == 'Jul2021' else x)
df4['QUARTER'] = df4['QUARTER'].apply(lambda x: '4Q21' if x == 'Dec2021' else '4Q21' if x == 'Nov2021' else '4Q21' if x == 'Oct2021' else x)
df4['QUARTER'] = df4['QUARTER'].apply(lambda x: '1Q22' if x == 'Jan2022' else '1Q22' if x == 'Feb2022' else '1Q22' if x == 'Mar2022' else x)
df4['QUARTER'] = df4['QUARTER'].apply(lambda x: '2Q22' if x == 'Jun2022' else '2Q22' if x == 'May2022' else '2Q22' if x == 'Apr2022' else x)
df4['QUARTER'] = df4['QUARTER'].apply(lambda x: '3Q22' if x == 'Sep2022' else '3Q22' if x == 'Aug2022' else '3Q22' if x == 'Jul2022' else x)
df4['QUARTER'] = df4['QUARTER'].apply(lambda x: '4Q22' if x == 'Dec2022' else '4Q22' if x == 'Nov2022' else '4Q22' if x == 'Oct2022' else x)

df5 = df4.loc[df4['QUARTER']=='2Q22',]
#df4 = df4.loc[df4['STATUS']!='Discarded',]
#df4 = df4.loc[df4['STATUS']!='Development',]
#df4 = df4.loc[df4['STATUS']!='Ready for development',]
#df4 = df4.loc[df4['STATUS']!='Specification',]
#df4 = df4.loc[df4['STATUS']!='Validation',]
#df4 = df4.loc[df4['STATUS']!='User acceptance testing (UAT)',]
#df5 = df4
df5.drop(columns=["ORIGIN RELEASE"],inplace=True)

filepath_ = r"C:\Users\016434613\Downloads\progress_dt_tasks_master_report.xlsx"

df_Development = pd.read_excel(filepath_,sheet_name="Development (Iterations)")

df_Development = df_Development.loc[:,['VERSION ID','DEVELOPERS']]
df_Development.drop_duplicates('VERSION ID',inplace=True)

df_Development.reset_index(inplace=True, drop=True)

df5 = df5.merge(df_Development, on="VERSION ID", how="left")

#df5['Developer'] = df5.apply(lambda x: x['Developer_x'] if str(x['Developer_y'])=="nan" else x['Developer_y'],axis=1)
#df5.drop(columns=['Developer_y','Developer_x'],inplace=True)

from urllib import request as req
import json

mgrserials = ['016434613','A01130693','A51733693','900688897']
mgrname = {'016434613':'Martin Campos','A01130693':'Marek Tarkos','A51733693':'Andrej Csiaki','900688897':'Melissa Bledsoe'}
devnames = {}
for mgr in mgrserials:
    request = req.urlopen("https://unified-profile-api.us-south-k8s.intranet.ibm.com/v3/profiles/" + mgr + "/teamResolved")
    datos = request.read().decode()
    dictdatos = json.loads(datos)
    devlist = dictdatos['content']['functional']['reports']
    for dev in devlist:
        #method setdefault(key,[]) sirve para poner una primera key con una lista vacia, si no, tira error de que no existe la key
        devnames.setdefault(mgrname[mgr], []).append(dev['nameDisplay'])
        
### Create Lists ###
devlistmc = devnames['Martin Campos']
devlistmt = devnames['Marek Tarkos']
devlistac = devnames['Andrej Csiaki']
devlistmb = devnames['Melissa Bledsoe']

### Create 2 dimension df, 1 column with the list, second column value fixed de Mgr ###
df6 = pd.DataFrame(devlistmc, columns=['DEVELOPERS'])
df6.loc[df6['DEVELOPERS']=="Pablo (Paul) Iacovone",'DEVELOPERS']="Pablo Ariel Iacovone"
df6.loc[df6['DEVELOPERS']=="Santiago Akira Kitashima",'DEVELOPERS']="Santiago Kitashima"
df6.loc[df6['DEVELOPERS']=="Ezequiel Ricardo Ferlauto",'DEVELOPERS']="Ezequiel Ferlauto"
df6['Manager'] = "Martin Campos"

df7 = pd.DataFrame(devlistmt, columns=['DEVELOPERS'])
#df7.loc[df7['DEVELOPERS']=="Viera Babčancová",'DEVELOPERS']="Viera Babcancova"
#df7.loc[df7['DEVELOPERS']=="Milan Babčanec",'DEVELOPERS']="Milan Babcanec"
#df7.loc[df7['DEVELOPERS']=="Adam Balák",'DEVELOPERS']="Adam Balak"
#df7.loc[df7['DEVELOPERS']=="Peter Blahut",'DEVELOPERS']="Peter Blahut"
#df7.loc[df7['DEVELOPERS']=="Viera Csiaki",'DEVELOPERS']="Viera Csiaki"
#df7.loc[df7['DEVELOPERS']=="Barbora Depešová",'DEVELOPERS']="Barbora Depesova"
#df7.loc[df7['DEVELOPERS']=="Richard Hantke",'DEVELOPERS']="Richard Hantke"
#df7.loc[df7['DEVELOPERS']=="Lenka Holáňová",'DEVELOPERS']="Lenka Holanova"
#df7.loc[df7['DEVELOPERS']=="Andrea Hrčková",'DEVELOPERS']="Andrea Hrckova"
#df7.loc[df7['DEVELOPERS']=="Miroslav Kocian",'DEVELOPERS']="Miroslav Kocian"
#df7.loc[df7['DEVELOPERS']=="Kristína Margetinová",'DEVELOPERS']="Kristina Margetinova"
#df7.loc[df7['DEVELOPERS']=="Lucia Mikuš",'DEVELOPERS']="Lucia Mikus"
#df7.loc[df7['DEVELOPERS']=="Soňa Olšová",'DEVELOPERS']="Sona Olsova"
#df7.loc[df7['DEVELOPERS']=="Marta Pancotti",'DEVELOPERS']="Marta Pancotti"
#df7.loc[df7['DEVELOPERS']=="Nina Tadlánková",'DEVELOPERS']="Nina Tadlankova"
#df7.loc[df7['DEVELOPERS']=="Ivan Trubač",'DEVELOPERS']="Ivan Trubac"
#df7.loc[df7['DEVELOPERS']=="Kristína Vojtová",'DEVELOPERS']="Kristina Vojtova"
df7.loc[df7['DEVELOPERS']=="Lubomir Cmiko (Lubo)",'DEVELOPERS']="Lubo Čmiko"
#df7.loc[df7['DEVELOPERS']=="Radomíra Šulová",'DEVELOPERS']="Radomira Sulova"
df7['Manager'] = "Marek Tarkos"

df8 = pd.DataFrame(devlistac, columns=['DEVELOPERS'])
#df8.loc[df8['DEVELOPERS']=="Marián Chrást",'DEVELOPERS']="Marian Chrast"
#df8.loc[df8['DEVELOPERS']=="Massimo Cireddu",'DEVELOPERS']="Massimo Cireddu"
#df8.loc[df8['DEVELOPERS']=="Katarina Kadlecova",'DEVELOPERS']="Katarina Kadlecova"
#df8.loc[df8['DEVELOPERS']=="Marta Kapolková",'DEVELOPERS']="Marta Kapolkova"
#df8.loc[df8['DEVELOPERS']=="Nikolaj Kniha",'DEVELOPERS']="Nikolaj Kniha"
#df8.loc[df8['DEVELOPERS']=="Veronika Krchnáková",'DEVELOPERS']="Veronika Krchnakova"
#df8.loc[df8['DEVELOPERS']=="Michele Lobina",'DEVELOPERS']="Michele Lobina"
#df8.loc[df8['DEVELOPERS']=="Vladimír Maťo",'DEVELOPERS']="Vladimir Mato"
#df8.loc[df8['DEVELOPERS']=="Veronika Miklovičová",'DEVELOPERS']="Veronika Miklovicova"
#df8.loc[df8['DEVELOPERS']=="Richard Nerodolík",'DEVELOPERS']="Richard Nerodolik"
#df8.loc[df8['DEVELOPERS']=="Micheál O'Reilly",'DEVELOPERS']="Mícheál O'Reilly"
#df8.loc[df8['DEVELOPERS']=="Michala Ožóg",'DEVELOPERS']="Michala Ozog"
#df8.loc[df8['DEVELOPERS']=="Branislav Podoba",'DEVELOPERS']="Branislav Podoba"
df8.loc[df8['DEVELOPERS']=="Andre Daniel Rodrigues",'DEVELOPERS']="Andre Rodrigues"
#df8.loc[df8['DEVELOPERS']=="Terézia Straková",'DEVELOPERS']="Terezia Strakova"
#df8.loc[df8['DEVELOPERS']=="Lenka Tarkošová",'DEVELOPERS']="Lenka Tarkosova"
#df8.loc[df8['DEVELOPERS']=="Valentína Vargová",'DEVELOPERS']="Valentina Vargova"
#df8.loc[df8['DEVELOPERS']=="Marek Štípala",'DEVELOPERS']="Marek Stipala"
df8['Manager'] = "Andrej Csiaki"

df9 = pd.DataFrame(devlistmb, columns=['DEVELOPERS'])
#df9.loc[df9['DEVELOPERS']=="Gabe Hutson",'DEVELOPERS']="Gabriel Hutson"
df9.loc[df9['DEVELOPERS']=="Nuno Macedo Gomes",'DEVELOPERS']="Nuno Filipe Macedo Gomes"
df9['Manager'] = "Melissa Bledsoe"


### Append results into one df - df10 ####
df10 = df6
df10 = df10.append(df7)
df10 = df10.append(df8)
df10 = df10.append(df9)

pos = []
valor = []
cont=0

for i in range(df5['DEVELOPERS'].size):
    x=re.search(",",df5['DEVELOPERS'][i])
    if x is not None:
        valor.append(df5['DEVELOPERS'][i])
        Devel = valor[cont].split(",")
        df5['DEVELOPERS'][i]=Devel[0]
        pos.append(i)
        cont+=1

df5 = df5.merge(df10, on="DEVELOPERS", how="left")
#df5.loc[df5["ROBOT_NUMBER"]=="ODA112","Developer"]="Bruno Secchiari"
#df5.loc[df5["ROBOT_NUMBER"]=="ODA127","Developer"]="Bruno Secchiari"
for t in range(len(pos)):
    df5['DEVELOPERS'][pos[t]]=valor[t]

df5.loc[df5['DEVELOPERS']=="Andre Daniel Rodrigues",'Manager']="Andrej Csiaki"
df5.loc[df5['DEVELOPERS']=="Martin Nicolas Benitez Alegre",'Manager']="Martin Campos"
df5.loc[df5['DEVELOPERS']=="Tomas Funes",'Manager']="Martin Campos"
df5.loc[df5['ROBOT_NUMBER']=="ODA139",'DEVELOPERS']="Ezequiel Ferlauto"
df5.loc[df5['ROBOT_NUMBER']=="ODA149",'DEVELOPERS']="Ezequiel Ferlauto"

df5['Count']=1
        
df5 = df5.loc[:,['Manager','DEVELOPERS','Count','SAVING HS','QUARTER','ROBOT_NUMBER','TITLE','STATUS','AUTOMATION TYPE','REGIONS']]
df5 = df5.rename(columns={'DEVELOPERS':'Developer'})
df5.to_excel(r"C:\Users\016434613\Desktop\Delivery Report.xlsx", index=False)
validsub()


end = time.time()
mins = (end-start)/60.00
if mins < 1:
    mins = end-start
    minsstr = "Report created in: {:.2f} seconds".format(mins) 
    print(minsstr)
else:
    minsstr = "Report created in: {:.2f} minutes".format(mins)
    print(minsstr)

Report created in: 11.31 seconds
